In [55]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random 
import pickle

In [2]:
print(tf.__version__)

1.4.0


In [3]:
train = pd.read_csv('../input/train.csv.zip')

In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [81]:
train.iloc[0,1]
print(train.columns)

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


In [6]:
toxic = train[train.toxic == 1]

In [7]:
print(train.shape, toxic.shape)

(159571, 8) (15294, 8)


In [8]:
toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [9]:
toxic.iloc[2,1]

"Bye! \n\nDon't look, come or think of comming back! Tosser."

In [10]:
toxic.iloc[3,1]

"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...\n\n1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!\n\n2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!\n\n3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!\n\nBeware of the Dark Side!"

In [112]:
class textCNN:
    
    def __init__(self,**params):
        # params is a dictionary
        self.params = params
        self.y = None
    
    def fit(self,X,y):
        # X is a list of comments. [[fxxk, ..], [something, ..]]
        # len(X) == N == len(y)
        # y is a numpy array (N, 6)
        self.X = np.array(X)
        self.y = y
        self._train()
    
    def predict(self,X):
        print('predict')
        # X is a list of comments. [[fxxk, ..], [something, ..]]
        self.X = X # Xt means X for test data
        return self.predictx()
    
    def predictx(self):
        tf.reset_default_graph()
        self.params['epochs'] = 1
        # build a tf computing graph
        logit = self._build()
        logit = tf.nn.sigmoid(logit)
        preds = []
        print('here')
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())
            self.load(sess)
            for c,Xb in enumerate(self._batch_gen(shuffle=False)):
                pred = sess.run(logit,feed_dict={self.inputs:Xb})
                if c%10 == 0:
                    print("batch %d predicted"%(c))
                preds.append(pred)
        preds = np.vstack(preds)
        return preds
        
    def _train(self):
        tf.reset_default_graph()
        # build a tf computing graph
        logit = self._build()
        label = tf.placeholder(dtype=tf.int32,shape=[None,6]) # B,classes
        losst = self.get_loss(logit,label)
        opt_op = self.get_opt(losst)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())
            self.load(sess)
            for c,(Xb,yb) in enumerate(self._batch_gen(shuffle=True)):
                loss,_ = sess.run([losst,opt_op],feed_dict={self.inputs:Xb, label:yb})
                if c%10 == 0:
                    print("batch %d train loss %.4f"%(c,loss))
                if c>100:
                    break
            self.save(sess)
        
    def get_opt(self,loss):
        opt = tf.train.AdamOptimizer(learning_rate=0.001)
        return opt.minimize(loss)
        
    def get_loss(self, logit, label):
        # build the loss tensor
        label = tf.cast(label,tf.float32)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,labels=label)
        return tf.reduce_mean(loss)
    
    def save(self, sess):
        varss = tf.trainable_variables()
        weights = {} # var.name => var.value: a numpy array
        for var in varss:
            val = sess.run(var)
            weights[var.name] = val
        pickle.dump(weights,open('weight.p','wb'))

    def load(self, sess, path = 'weight.p'):
        weights = pickle.load(open(path,'rb'))
        varss = tf.trainable_variables()
        for var in varss:
            value = weights[var.name]
            assign_op = var.assign(value)
            sess.run(assign_op)
        
    def _build(self):
        
        # B is the batch size
        # S is the sequence length
        # E is the embedding size: 16 by default
        # V is the vocab size
        # embedding space will be: V x E
        # self.inputs => Xb
        E = self.params.get('embedding_size',16)
        V = self.params['vocab_size']
        
        netname = 'textCNN'
        self.inputs = tf.placeholder(dtype=tf.int32,shape=[None,None]) # B,S
        
        with tf.variable_scope(netname):
            # embedding lookup
            print("Input:",self.inputs.get_shape().as_list())
            net = self.embedding_lookup(E, V, self.inputs,reuse=False) # dim(net) = B,S,E
            print("after embedding lookup:",net.get_shape().as_list())
            
            net1 = self.conv1d(net,name='conv1',ngrams=3,stride=1,cin=E,nfilters=30)
            net2 = self.conv1d(net,name='conv2',ngrams=1,stride=1,cin=E,nfilters=10)
            net3 = self.conv1d(net,name='conv3',ngrams=5,stride=1,cin=E,nfilters=50)
            
            print("before maxpool net1: {} net2: {} net3: {}".format(net1.get_shape().as_list(),net2.get_shape().as_list(),net3.get_shape().as_list()))
            
            net1 = tf.reduce_max(net1,axis=1)
            net2 = tf.reduce_max(net2,axis=1)
            net3 = tf.reduce_max(net3,axis=1)
            
            print("after maxpool net1: {} net2: {} net3: {}".format(net1.get_shape().as_list(),net2.get_shape().as_list(),net3.get_shape().as_list()))
           
            
            net  = tf.concat([net1,net2,net3],axis=1)
            print("after concat:",net.get_shape().as_list())
            #print("after conv1d:",net.get_shape().as_list())
            
            net = self.fc(net, 'fc', cin=90, cout=6, activation=None)
            print("after fc:",net.get_shape().as_list())
            return net
            
    def fc(self, net, name, cin, cout, activation='relu'):
        weights = tf.get_variable(name=name,shape=[cin,cout],dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer())
        net = tf.matmul(net,weights)
        if activation == 'relu':
            net = tf.nn.relu(net)
        return net
        
    def conv1d(self, net, name, ngrams, stride, cin, nfilters,activation='relu'):
        # filters: kernel_size, cin, cout
        filters = tf.get_variable(name='%s_filters'%name,shape=[ngrams,cin,nfilters],dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer()) # filter variable
        net = tf.nn.conv1d(net, filters, stride, padding='VALID')
        if activation == 'relu':
            net = tf.nn.relu(net)
        return net
    
    def embedding_lookup(self,E,V,x,reuse=False):
        # x is the key
        with tf.variable_scope('embedding', reuse=reuse):
            
            # initialize the embedding matrix variable
            w = tf.get_variable(name='w',shape=[V-1,E],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer()) # embedding matrix
            # w.name = 'textCNN/embedding/w:0'
            self.em = w
            c = tf.zeros([1,E])
            w = tf.concat([c,w],axis=0)
            
            
            # embedding lookup
            x = tf.cast(x,tf.int32)
            x = tf.nn.embedding_lookup(w, x, name='word_vector')  # (N, T, M) or (N, M)
            return x
    
    def _batch_gen(self, shuffle=True):
        X,y = self.X, self.y
        B = self.params.get('batch_size', 128)
        epochs = self.params.get('epochs', 10)
        ids = [i for i in range(len(X))]
        batches = len(X)//B + 1
        print(epochs,batches)
        for epoch in range(epochs):
            if shuffle:
                random.shuffle(ids)
            for i in range(batches):                
                idx = ids[i*B:(i+1)*B]
                Xb = self.padding(X[idx])
                if y is not None:
                    yield Xb,y[idx]
                else:
                    yield Xb
            if (i+1)*B < len(X):
                idx = ids[(i+1)*B:len(X)]
                Xb = self.padding(X[idx])
                yield Xb
    
    def padding(self,X):
        Xn = []
        maxlen = max([len(i) for i in X])
        for x in X:
            xn = x+[0]*(maxlen-len(x))
            assert len(xn)==maxlen
            Xn.append(xn)
        return np.array(Xn)
    
    def _predict(self):
        pass

In [12]:
# construct a vocabulary as a dictionary: {word: integer}
vocab = {'__none__':0}
for index, row in train.iterrows():
    comment = row['comment_text']
    if index >0 and index%50000 ==0:
        print('%d rows done, size of vocab = %d'%(index,len(vocab)))
    words = comment.split()
    for word in words:
        if word not in vocab:
            vocab[word] = len(vocab)
            

50000 rows done, size of vocab = 237044
100000 rows done, size of vocab = 384696
150000 rows done, size of vocab = 509860


In [13]:
# map the word to an integer for all the words
# build the X
X = []
for index, row in train.iterrows():
    comment = row['comment_text']
    if index >0 and index%50000 ==0:
        print('%d rows done'%(index))
    words = comment.split()
    x = []
    for word in words:
        x.append(vocab.get(word, 0))
    X.append(x)

50000 rows done
100000 rows done
150000 rows done


In [85]:
test = pd.read_csv('../input/test.csv.zip')
Xt = []
for index, row in test.iterrows():
    comment = row['comment_text']
    if index >0 and index%50000 ==0:
        print('%d rows done'%(index))
    words = comment.split()
    x = []
    for word in words:
        x.append(vocab.get(word, 0))
    Xt.append(x)

50000 rows done
100000 rows done
150000 rows done


In [96]:
classes = train.columns.values[2:]
Xt = np.array(Xt)

In [18]:
y = train[classes].values

In [19]:
print(len(X),len(y),y.shape)

159571 159571 (159571, 6)


In [119]:
model = textCNN(vocab_size=len(vocab),embedding_size=4)

In [74]:
model.fit(X,y)

Input: [None, None]
after embedding lookup: [None, None, 4]
before maxpool net1: [None, None, 30] net2: [None, None, 10] net3: [None, None, 50]
after maxpool net1: [None, 30] net2: [None, 10] net3: [None, 50]
after concat: [None, 90]
after fc: [None, 6]
batch 0 train loss 0.6932
batch 10 train loss 0.6878
batch 20 train loss 0.6757
batch 30 train loss 0.6538
batch 40 train loss 0.6171
batch 50 train loss 0.5593
batch 60 train loss 0.4951
batch 70 train loss 0.4168
batch 80 train loss 0.3404
batch 90 train loss 0.2365
batch 100 train loss 0.2042


In [77]:
model.fit(X,y)

Input: [None, None]
after embedding lookup: [None, None, 4]
before maxpool net1: [None, None, 30] net2: [None, None, 10] net3: [None, None, 50]
after maxpool net1: [None, 30] net2: [None, 10] net3: [None, 50]
after concat: [None, 90]
after fc: [None, 6]
batch 0 train loss 0.1849
batch 10 train loss 0.1930
batch 20 train loss 0.1750
batch 30 train loss 0.1424
batch 40 train loss 0.1465
batch 50 train loss 0.1788
batch 60 train loss 0.1234
batch 70 train loss 0.1420
batch 80 train loss 0.1780
batch 90 train loss 0.1153
batch 100 train loss 0.1293


In [117]:
model = textCNN(vocab_size=len(vocab),embedding_size=4)
yp = model.predict(Xt)
yp.shape
s = pd.DataFrame(yp,columns = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate'])
s['id'] = test['id']

In [118]:
s.to_csv('submission.csv',index=False)

In [67]:
weights = pickle.load(open('weight.p','rb'))

In [69]:
for name,values in weights.items():
    print(name,values.shape)

textCNN/conv1_filters:0 (3, 4, 30)
textCNN/conv2_filters:0 (1, 4, 10)
textCNN/embedding/w:0 (532299, 4)
textCNN/conv3_filters:0 (5, 4, 50)
textCNN/fc:0 (90, 6)


In [78]:
varss = tf.trainable_variables()

In [80]:
type(varss)

list